In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Gaussian Elimination with Partial Pivoting

## Question 1
Write a function called `solve_system_gauss()` that takes in a matrix `A` and a vector `b` and solves the matrix equation $A\bf{x} = \bf{b}$ using Gaussian elimination (row reduction) with partial pivoting and back substitution. Outside of `solve_system_gauss()`, you should have the functions:
* `reduce_row()`
* `row_swap()`
* `back_sub()`

Each of these functions performs one part of solving the system of equations with Gaussian elimination

## Solution

In [2]:
def Reduce_Row(A, row_index):
    
    for i in range(row_index+1,np.shape(A)[0]):                 # iterate over the rows of A below the current row
        c = np.divide( A[i,row_index], A[row_index,row_index] ) # define a coefficient for Gaussian Elimnation
        A[i,:] = A[i,:] - c*A[row_index,:]                      # use Gaussian Elimination to assign row values
        
    return

In [3]:
def Row_Swap(A, row_index):
   
    n = np.shape(A)[1]                                       # define the row length of A
    temp = np.zeros(n)                                       # intialize a dummy row
    current_column = np.array(A[row_index:,row_index])       # define the current column
    max_row = A[row_index+np.argmax(abs(current_column)), :] # define the row with the maximum entry in the current column
    current_row = A[row_index,:]                             # define the current row
    temp[:] = current_row[:]                                 # assign the current row to temp
    current_row[:] = max_row[:]                              # assign the max row to the current row
    max_row[:] = temp[:]                                     # assign temp (current row) to the max row
    
    return

In [4]:
def Back_Sub(U,b):
    
    x = np.zeros((np.shape(U)[0],1))                    # initialize vector
    for row in range(np.shape(U)[0]-1,-1,-1):           # update x_row from the last row to the first row
        x[row] = b[row]                                 # start with x_row = b_row
        for column in range(row+1, np.shape(U)[1]):     # iterate over entries in row after x_row
            x[row] = x[row] - U[row,column] * x[column] # subtract each entry other than x_row
        x[row] /= U[row,row]                            # divide by coefficient of x_row
        
    return x    

In [5]:
def Solve_System_Gauss(A,b):
    
    aug = np.hstack( (A,b) )                          # augment A with b
    for column in range(np.shape(A)[1]):              # iterate of the rows of A
        Row_Swap(aug,column)                          # partial pivot
        Reduce_Row(aug,column)                        # Gaussian Elimination
    aug_mat, aug_b = np.hsplit(aug, [np.shape(A)[1]]) # extract the matrix part and the constant vector part from the augmented matrix
    sol = Back_Sub(aug_mat,aug_b)                     # back-substitue the matrix and vector after row reduction
    
    return sol

## Question 2
Use `solve_system_gauss()` to solve the system of equations:
\begin{align*}
 0 x_{1}+ -2 x_{2}+ 0 x_{3}+ 0 x_{4}+ 0 x_{5}&= 4 \\ 
 -6 x_{1}+ 0 x_{2}+ 4 x_{3}+ 0 x_{4}+ 0 x_{5}&= -36 \\ 
 2 x_{1}+ 8 x_{2}+ 4 x_{3}+ 0 x_{4}+ -2 x_{5}&= -12 \\ 
 -6 x_{1}+ -16 x_{2}+ -8 x_{3}+ -2 x_{4}+ 4 x_{5}&= 24 \\ 
 -2 x_{1}+ -6 x_{2}+ 2 x_{3}+ 0 x_{4}+ 0 x_{5}&= -2
\end{align*}

## Solution

In [6]:
A = np.array([[ 0,-2., 0, 0, 0], 
              [-6,  0, 4, 0, 0], 
              [ 2,  8, 4, 0,-2], 
              [-6,-16,-8,-2, 4], 
              [-2, -6, 2, 0, 0]])

b = np.array([[ 4.],
              [-36], 
              [-12], 
              [ 24], 
              [ -2]])

Solve_System_Gauss(A, b)

array([[ 4.],
       [-2.],
       [-3.],
       [-4.],
       [-4.]])

# A theoretical and practical problem with Gaussian Elimination

## Question 1
Consider the system of equations
\begin{align*}
\varepsilon x_1 + x_2 & = 1 \\
x_1 + x_2 & = 2
\end{align*}
Find a formula for both components of the solution $x_1$ and $x_2$ expressed in terms of $\varepsilon$. What is the solution as $\varepsilon \to 0$?

## Solution

The given system can be solved with an augmented matrix as follows:

$ \begin{bmatrix*}
\varepsilon & 1 &|& 1 \\
1 & 1 &|& 2 
\end{bmatrix*}
\quad \Rightarrow \quad
$
$
R_1 \leftrightarrow R_2
\quad
\begin{bmatrix}
1 & 1 &|& 2 \\
\varepsilon & 1 &|& 1
\end{bmatrix}
\quad \Rightarrow \quad
$
$
R_2 \rightarrow R_2 - \varepsilon R_1
\quad
\begin{bmatrix}
1 & 1 &|& 2 \\
0 & 1-\varepsilon &|& 1 - 2\varepsilon
\end{bmatrix}
\quad
\Rightarrow
\quad
$
$
\begin{align}
x_1 + x_2 &= 2 \\
(1 - \varepsilon)x_2 &= 1 - 2\varepsilon
\end{align}
\quad
\Rightarrow
\quad
$
$
\begin{align}
x_1 &= \frac{(2 - 2\varepsilon) - (1 - 2\varepsilon)}{1 - \varepsilon} = \frac{1}{1 - \varepsilon} \\
x_2 &= \frac{1 - 2\varepsilon}{1 - \varepsilon}
\end{align}
$

$$
\lim_{\varepsilon\to0} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} = \begin{bmatrix} 1 \\ 1 \end{bmatrix}
$$

## Question 2
Solve the above system of equations WITHOUT partial pivoting, where $\varepsilon = \varepsilon_1 = 10^{-6}$. Solve the same system WITHOUT partial pivoting, where $\varepsilon = \varepsilon_2 = 10^{-16}$. Comment on the results. It may help to add an argument to `solve_system_gauss(...,pivot=True)`, where the input parameter `pivot` is a boolean whose default value is `True`, but a user (i.e., you) could instead specify `pivot=False` when calling `solve_system_gauss()`.

## Solution
Let's redefine `Solve_System_Gauss` to allow toggling of partial pivoting.

In [7]:
def Solve_System_Gauss(A,b, pivot=True):
    
    aug = np.hstack( (A,b) )                            # augment A with b
    for row in range( np.shape(aug)[0] ):               # iterate over the rows of A
        if pivot:                                       # check if partial pivoting will be used
            Row_Swap( aug, row )                    # partial pivot the augment matrix in the current row
        Reduce_Row( aug, row )                          # reduce the augmented matrix with Gaussian Elimination
    aug_mat, aug_b = np.hsplit( aug, [np.shape(A)[0]] ) # seperate the augmented matrix back into a coefficient and constant matrix
    sol = Back_Sub( aug_mat, aug_b )                    # define the system solution
    
    return sol

Now let's plug in our system with $\varepsilon = 10^{-6}$ and with $\varepsilon = 10^{-16}$.

In [8]:
e = 10**-6
test_A = np.array([[e, 1],
                   [1, 1]])

test_b = np.array([[1],
                   [2]])

print('Epsilon = 10**-6 :', Solve_System_Gauss(test_A,test_b,pivot=False))

e = 10**-16
test_A = np.array([[e, 1],
                   [1, 1]])

test_b = np.array([[1],
                   [2]])

print('Epsilon = 10**-16 :', Solve_System_Gauss(test_A,test_b,pivot=False))

Epsilon = 10**-6 : [[1.000001]
 [0.999999]]
Epsilon = 10**-16 : [[2.22044605]
 [1.        ]]


When $\varepsilon = 10^{-6}$, the solution is very close to the limit $\begin{bmatrix} 1 \\ 1 \end{bmatrix}$ but still off by $10^-6$.  When $\varepsilon = 10^{-16}$ the solution component $x_2$ is fine, but the component $x_1$ is very far off. These errors occur because of a lack of partial pivotting which leads to catastrophic cancellation. The first pivot is VERY small compared to the rest of the matrix, so some error is introduced when $x_1$ is rounded to 1. This error is then multiplied by $10^{16}$ when $\varepsilon$ is divided over.

## Question 3
Solve the same system WITH partial pivoting, again once each time for $\varepsilon = \varepsilon_1$ and $\varepsilon = \varepsilon_2$. Compare and explain your results with those obtained without partial pivoting.

## Solution


In [9]:
e = 10**-6
test_A = np.array([[e, 1],
                   [1, 1]])

test_b = np.array([[1],
                   [2]])

print('Epsilon = 10**-6 :', Solve_System_Gauss(test_A,test_b,pivot=True))

e = 10**-16
test_A = np.array([[e, 1],
                   [1, 1]])

test_b = np.array([[1],
                   [2]])

print('Epsilon = 10**-16 :', Solve_System_Gauss(test_A,test_b,pivot=True))

Epsilon = 10**-6 : [[1.000001]
 [0.999999]]
Epsilon = 10**-16 : [[1.]
 [1.]]


With partial pivoting, $|1| \gg |\varepsilon| $, so there is virtually no error that could introduce catastrophic cancellation.

# Solving Systems with LU Decomposition

## Question 1
Write a function called `solve_system_LU()` that takes in a matrix `A` and a vector `b` to solve the equation $A\mathbf{x} = \mathbf{b}$. This function should call three functions:
* `my_LU()`
* `forward_sub()`
* `back_sub()`

Each of these functions performs one step required to solve $A\mathbf{x} = \mathbf{b}$ via an LU decomposition.

## Solution

In [10]:
def My_LU(A):

    L = np.eye(np.shape(A)[0])                                           # initialize L as the identity matrix
    U = np.array(A)                                                      # initialize U as input matrix
    for column in range(np.shape(U)[1]):                                 # iterate over the columns of U 
        for row in range(column+1, np.shape(U)[0]):                      # iterate through the rows of the current column past the pivot
            L[row,column] = np.divide( U[row,column], U[column,column] ) # update the corresponding entry of L
        Reduce_Row(U,column)                                             # update the current column of U

    return L, U
    

In [11]:
def Forward_Sub(L,b):
   
    x = np.zeros((np.shape(L)[0]))                      # initialize solution vector
    for row in range(np.shape(L)[0]):                   # update x_row from the first row to the last row
        x[row] = b[row]                                 # start with x_row = b_row
        for column in range(row):                       # iterate over entries in row before x_row
            x[row] = x[row] - L[row,column] * x[column] # subtract each entry other than x_row
        x[row] /= L[row,row]                            # divide by coefficient of x_row
        
    return x

In [12]:
def Back_Sub(U,b):
   
    x = np.zeros((np.shape(U)[0]))                      # initialize solution vector
    for row in range(np.shape(U)[0]-1,-1,-1):           # update x_row from the last row to the first row
        x[row] = b[row]                                 # start with x_row = b_row
        for column in range(row+1, np.shape(U)[1]):     # iterate over entries in row after x_row
            x[row] = x[row] - U[row,column] * x[column] # subtract each entry other than x_row
        x[row] /= U[row,row]                            # divide by coefficient of x_row
    
    return x    

In [13]:
def Solve_System_LU(A,b):
    
    L, U = My_LU(A)      # perform LU decomposition on A
    y = Forward_Sub(L,b) # Ly = b
    x = Back_Sub(U,y)    # Ux = y

    return x

## Question 2
Solve the system 
\begin{align*}
x_1 - 2x_2 - 4x_3 - 3x_4 & = 1 \\
2x_1 -7x_2 - 7x_3 - 6x_4 & = 7 \\
-x_1 + 2x_2 + 6x_3 + 4x_3& = 0 \\
-4x_1 - x_2 + 9x_3 + 8x_4& = 3
\end{align*}
using `solve_system_LU()`

## Solution

In [14]:
test_A = np.array([[ 1.,-2,-4,-3],
                   [ 2,-7,-7,-6],
                   [-1, 2, 6, 4],
                   [-4,-1, 9, 8]])

test_b = np.array([[1.],
                   [7],
                   [0],
                   [3]])

Solve_System_LU(test_A, test_b).reshape(np.shape(test_A)[0],1)

<ipython-input-11-aae961225af9>:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x[row] = b[row]                                 # start with x_row = b_row


array([[-2.],
       [-1.],
       [ 2.],
       [-3.]])

# Inverting a matrix using LU decomposition

### Question 1
The way you learned to invert a matrix $A\in\mathbb{R}^{n\times n}$ in a first course in Linear Algebra is to row-reduce the augmented matrix
\begin{equation}
[A \ \ \vert \ \ I_n]
\end{equation}
until the matrix $A$ was reduced to the identity matrix. The matrix that resulted from applying the row-reduction steps for $A$ to $I_n$ gave the inverse of $A$.

Write a Python function `my_inv_LU` that uses the $LU$ decomposition to invert a matrix A. When grading the problem, I will carefully consider the *efficiency* of your inversion routine.

Of note, many linear algebra softwares invert a matrix via LU decomposition with pivoting (we haven't done pivoting in the $LU$ decomposition, but the point remains)

## Solution

In [15]:
def My_Inv_LU(A):

    I = np.eye(np.shape(A)[0])                           # define identity matrix
    A_inv = np.zeros( (np.shape(A)[0], np.shape(A)[1]) ) # initialize inverse matrix
    n = np.shape(A_inv)[1]                               # define number of columns of A
    L, U = My_LU(A)                                      # perform LU decomposition ONCE on A
    for col in range(n):                                 # iterate over the columns of A
        y = Forward_Sub(L, I[:,col])                     # solve Ly = I[:, col]
        A_inv[:,col] = Back_Sub(U, y)                    # solve UA_inv[:, col] = y
        
    return A_inv

## Question 2
Solve the $4\times 4$ system in the previous question by finding $A^{-1}$ with `my_inv_LU` and the built-in function `np.dot()`.

## Solution

In [16]:
test_A = np.array([[ 1.,-2,-4,-3],
                   [ 2,-7,-7,-6],
                   [-1, 2, 6, 4],
                   [-4,-1, 9, 8]])

test_b = np.array([[1.],
                   [7],
                   [0],
                   [3]])

A_inv = My_Inv_LU(test_A)
print(A_inv)
print(np.dot(A_inv, test_b))

[[14.66666667 -2.66666667  5.66666667  0.66666667]
 [-1.66666667  0.16666667 -0.66666667 -0.16666667]
 [-7.          1.5        -2.         -0.5       ]
 [15.         -3.          5.          1.        ]]
[[-2.]
 [-1.]
 [ 2.]
 [-3.]]


# Jacobi iteration

## Question 1
Write a function `my_jacobi()` that solves the matrix equation $A\mathbf{x} = \mathbf{b}$ with Jacobi iteration.

## Solution

In [17]:
# Ax = b
# C[i,j] = - A[i,j] / A[i,i]
# d[i] = b[i] / A[i,i]
# Then x^{k+1} = Cx^{k} + d, and x^{0} = d is a good guess
# Jacobi's method converges to solution x^{*} if A is row-diagonally dominant 
def My_Jacobi(A,b,N):
    
    C = np.zeros( (np.shape(A)[0], np.shape(A)[1]) )
    d = np.zeros(np.shape(A)[0])
    for i in range(np.shape(A)[0]):
        d[i] = np.divide( b[i], A[i,i] )
        for j in range(np.shape(A)[1]):
            if i!=j:
                C[i,j] = -np.divide( A[i,j], A[i,i] )
    x = np.zeros(np.shape(C)[1])
    for n in range(N):
        x = np.dot( C, x ) + d
        
    return x
        

## Question 2
Consider the system
\begin{align*}
3x_1 - x_2 & = 4 \\
-x_1 + 3x_2 - 1x_3 & = -6 \\
-x_2 + 3x_3 - x_4 & = 9 \\
-x_3 + 3x_4 & = -8
\end{align*}
Explain why this system is row-diagonally-dominant. Explain why this matters for Jacobi iteration. Solve the above system using `my_jacobi()` with 5 iterations. The actual answer is $[1, -1, 2, -2]$. How close is your answer?

## Solution

A matrix is row-diagonally-dominant if $|A_{i,i}| > \sum|A_{i,j}|$ where $j \neq i$. The coefficient matrix of the given system is $\begin{bmatrix} 3 & -1 & 0 & 0 \\ -1 & 3 & -1 & 0 \\ 0 & -1 & 3 & -1 \\ 0 & 0 & -1 & 3 \end{bmatrix}$. In each row $|A_{i,i}| = 3$, and $\sum|A{i,j}| \leq 2$ as $|-1|+|-1| = 2$ and $|-1| = 1$. Since the corresponding coefficient matrix of the system is row-diagonally-dominant, the system is too.

This matters for Jacobi iteration because convergence to the true solution is guarunteed regardless of the initial guess for row-diagonally-dominant systems.

In [18]:
test_A = np.array([[ 3,-1, 0, 0],
                   [-1, 3,-1, 0],
                   [ 0,-1, 3,-1],
                   [ 0, 0,-1, 3]])

test_b = np.array([[ 4],
                   [-6],
                   [ 9],
                   [-8]])

My_Jacobi(test_A, test_b, 5)

<ipython-input-17-3c73a640714e>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  d[i] = np.divide( b[i], A[i,i] )


array([ 1.04526749, -1.08641975,  2.07407407, -2.05349794])

My answer is within 0.1 for each component, which is okay for only 5 iterations.

## Question 3
Consider the system
\begin{align*}
2x_1 - x_2 & = 3 \\
-x_1 + 2x_2 - 1x_3 & = -5 \\
-x_2 + 2x_3 - x_4 & = 7 \\
-x_3 + 2x_4 & = -6
\end{align*}
Explain why this system is NOT row-diagonally-dominant. Peform 5 Jacobi iterations to solve this system. The actual answer is $[1,-1,2,-2]$. How close is your answer? Try again, but perform 30 iterations. How close is your answer?

## Solution

This system is not row-diagonally-dominant because the absolute values of diagonal entries are 2, and the sum of the absolute values of the other entries in rows 2 and 3 is 2. So the diagonal entries are not much larger than the rest of the row for rows 2 and 3. 

In [19]:
test_A = np.array([[ 2,-1, 0, 0],
                   [-1, 2,-1, 0],
                   [ 0,-1, 2,-1],
                   [ 0, 0,-1, 2]])

test_b = np.array([[ 3],
                   [-5],
                   [ 7],
                   [-6]])

My_Jacobi(test_A, test_b, 5)

<ipython-input-17-3c73a640714e>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  d[i] = np.divide( b[i], A[i,i] )


array([ 1.34375, -1.65625,  2.5625 , -2.40625])

This answer is much further off than before. An acceptable estimate should be accurate to at least one significant figure and usually much more.

In [20]:
My_Jacobi(test_A, test_b, 30)

<ipython-input-17-3c73a640714e>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  d[i] = np.divide( b[i], A[i,i] )


array([ 0.99797129, -0.99719639,  1.99671748, -1.99826728])

With 30 iterations then answer is much better, even better than the row-diagonally-dominant estimation but with 5 iterations which isn't too surprising.

## Question 4
What do you think Questions 2 and Questions 3 are trying to demonstrate?

## Answer

Questions 2 and 3 are trying to demonstrate the exact significance of row-diagonally-dominant matrices as they relate to Jacobi iteration. Convergence is guarunteed for these special matrices, but that doesn't necessarily mean that a non-special matrix won't converge. It may just converge slower than the special matrix.